# torch.use_deterministic_algorithms

A simple tool to help write reproducible PyTorch applications.

## Why do we care about reproducibility?

PyTorch is often used to train and run models to make decisions. Imagine that you create a model that can sometimes make different decisions given the same exact initial state (same platform, evironment, inputs, random seeds, etc.). Furthermore, imagine that each time you train the model with the same set of training data, the trained weights in the model are slightly different each time. In an extreme example of a self-driving car, this extra variability could increase the chances of an accident.

Simply put, a reproducible application is a better application. (However, in many cases, the benefits may not outweigh the performance costs. It depends on the application.)

## What kinds of algorithms are nondeterministic?

There are many examples, but I'll explain one: an algorithm that adds a bunch of floating point numbers together. If the numbers are summed in a different order each time, we can get a different result.

Below, we calculate the total sum of all the numbers in `a` and `b` three different ways. Each method gives a slightly different result.


In [1]:
a = [1e-30] * 10000
b = [1e-29] * 10000

separate_sum = sum(a) + sum(b)
combined_sum = sum(a + b)
interleaved_sum = sum([a_el + b_el for a_el, b_el in zip(a, b)])

print(separate_sum)
print(combined_sum)
print(interleaved_sum)

1.100000000000168e-25
1.1000000000002118e-25
1.0999999999998715e-25


One way nondeterministic summing could happen in a real application is if a large calculation is split up into multiple threads. Both the way the work is split up and the way the results are combined at the end could vary if those decisions are left to the system software. For instance, to save time, the system may decide to combine a given thread's result as soon as it is available, rather than prescribing a specific order.


## How do we guarantee reproducibility?

The pedantic answer is: we actually can't! The only way to be 100% confident that an operation is reproducible is to run it an infinite number of times with every possible input and show that it gives a consistent result every time. Unfortunately, we only have a finite amount of time. 

Instead of guaranteeing that an operator is deterministic, we can tell users specifically which operators we know are nondeterministic.

In PyTorch we know that an operation is nondeterministic if either:
* Someone has reported seeing the operation act nondeterministically after following the [Reproducibility guide](https://pytorch.org/docs/master/notes/randomness.html)
* We carefully look at the code for an operation and see that it uses an algorithm known to be nondeterminisic

## Enter `torch.use_deterministic_algorithms`

When this setting is turned on, all operations that use nondeterministic algorithms will either switch to a deterministic implementation or raise an error when they are called.

The documentation contains a list of all affected operations and the conditions under which they are affected.

[Link to documentation](https://pytorch.org/docs/master/generated/torch.use_deterministic_algorithms.html)

In [2]:
%%html
<iframe src="https://pytorch.org/docs/master/generated/torch.use_deterministic_algorithms.html" width="800" height="500"></iframe>

## Example (raise error)

As the above documentation shows, `kthvalue` is one of the operations that throws an error if `use_deterministic_algorithms` is turned on. We can see its nondeterministic behavior below. If the k-th value has duplicates, the index of the result can be any one of the duplicates.

In [3]:
import torch
torch.use_deterministic_algorithms(False) # False is the default state
torch.manual_seed(0)

a = torch.zeros(10000, device='cuda')

for _ in range(10):
    # Seed each time, just to show that nondeterminism does not come from RNG
    torch.manual_seed(0)
    
    res = a.kthvalue(1)
    
    # Index can be different each time
    print(f'value: {res.values}, index: {res.indices}')
    

value: 0.0, index: 16
value: 0.0, index: 80
value: 0.0, index: 112
value: 0.0, index: 240
value: 0.0, index: 80
value: 0.0, index: 464
value: 0.0, index: 80
value: 0.0, index: 80
value: 0.0, index: 16
value: 0.0, index: 112


If we turn `use_deterministic_algorithms` on, we get an error.

In [4]:
torch.use_deterministic_algorithms(True)
a.kthvalue(1)

/work2/kurtamohler/development/pytorch-det-complex-autograd/torch/__init__.py:470: UserWarning: torch.use_deterministic_algorithms is in beta, and its design and functionality may change in the future. (Triggered internally at  ../aten/src/ATen/Context.cpp:69.)
  _C._set_deterministic_algorithms(mode)


RuntimeError: kthvalue CUDA does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True)'. You can turn off determinism just for this operation if that's acceptable for your application. You can also file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation.

## Example (run alternate deterministic algorithm)

`torch.bmm` is one of the operations that runs an alternate deterministic algorithm when `use_deterministic_algorithms` is turned on. Below, we have a test that exercises the nondeterministic behavior multiple times and compares the results each time.

In [5]:
def run_bmm_test(num_iters=100):
    first_res = None

    for iter_num in range(num_iters):
        # Seed RNG to ensure we have the same data each time
        torch.manual_seed(0)

        a = torch.randn(100, 100, 100, device='cuda').to_sparse()
        b = torch.randn(100, 100, 100, device='cuda')

        res = torch.bmm(a, b)

        if first_res is None:
            first_res = res.clone()
        else:
            if not res.eq(first_res).all():
                print(f'Result mismatch after iteration {iter_num}')
                return
    print(f'Results match after all {num_iters} iterations')

When `use_deterministic_algorithms` is turned off, we get a mismatch after only the second iteration of the test.

In [6]:
torch.use_deterministic_algorithms(False)
run_bmm_test()

Result mismatch after iteration 1


But when it's turned on, results match each time

In [7]:
torch.use_deterministic_algorithms(True)
run_bmm_test()

Results match after all 100 iterations


## How does `torch.use_deterministic_algorithms` detect nondeterminism?

Nothing fancy. We just hard code it into each operation.

For raising an error, we have an internal function call. From [torch.kthvalue](https://github.com/pytorch/pytorch/blob/14282232d94032e2b6cff2b6cb044e53b95f4196/aten/src/ATen/native/cuda/Sorting.cu#L413):

```cpp
    at::globalContext().alertNotDeterministic("kthvalue CUDA");
```
And each operation that has an alternate deterministic implementation must check the flag's state and react appropriately.

From [torch.bmm](https://github.com/pytorch/pytorch/blob/14282232d94032e2b6cff2b6cb044e53b95f4196/aten/src/ATen/native/sparse/cuda/SparseCUDATensorMath.cu#L892):

```cpp
    deterministic = deterministic || globalContext().deterministicAlgorithms();
    cusparseSpMMAlg_t mm_alg = deterministic ? CUSPARSE_COOMM_ALG2 : CUSPARSE_COOMM_ALG1;
    ...
    TORCH_CUDASPARSE_CHECK(cusparseSpMM(..., mm_alg, ...));
```

## Enforcing documentation consistency

Documentation for nondeterministic algorithms is not updated automatically, so we're depending on contributors to update the docs for `torch.use_deterministic_algorithms` appropriately any time they change its behavior with respect to any operation.

Currently, the solution to this problem is to include a note next to every `at::globalContext().alertNotDeterministic` and `globalContext().deterministicAlgorithms` call in the codebase. The note points to internal documentation explaining the rules for using those API calls.

For `at::globalContext().alertNotDeterministic`: [Note [Writing Nondeterministic Operations]](https://github.com/pytorch/pytorch/blob/14282232d94032e2b6cff2b6cb044e53b95f4196/aten/src/ATen/Context.h#L148)

For `at::globalContext().deterministicAlgorithms`: [Note [Enabling Deterministic Operations]](https://github.com/pytorch/pytorch/blob/14282232d94032e2b6cff2b6cb044e53b95f4196/aten/src/ATen/Context.h#L119)